# Setup

### Import necessary modules and do some basic setup.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# TensorFlow ≥2.0 is required
import tensorflow_addons as tfa
import tensorflow as tf
assert tf.__version__ >= '2.0'

from tensorflow import keras
from tensorflow.keras import layers

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Common imports
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import dask
import datetime
import math
dask.config.set({'array.slicing.split_large_chunks': False})

# To make this notebook's output stable across runs
np.random.seed(42)

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Dotenv
from dotenv import dotenv_values

# Custom utils
from utils.utils_data import *
from utils.utils_ml import *

### Define some paths and constants.

In [ ]:
config = dotenv_values(".env")

# Paths
PATH_ERA5 = config['PATH_ERA5']
PATH_EOBS = config['PATH_EOBS']

# Some constants
G = 9.80665 
DATE_START = '1979-01-01'
DATE_END = '2020-12-31'
YY_TRAIN = [1979, 2015]
YY_TEST = [2016, 2020]
LEVELS = [500, 850, 1000]
LONS_INPUT = [-40, 30]
LATS_INPUT = [30, 80]
LONS_PREC = [-12, 30]
LATS_PREC = [32, 72]

# Data preparation

## Target variable: precipitation field

In [ ]:
# Precipitation ERA5
pr = get_era5_data(PATH_ERA5 + '/precipitation/day_grid1/*nc', DATE_START, DATE_END, LONS_PREC, LATS_PREC)

# Define precipitation extremes using the 95th percentile
#pr95 = precip_exceedance_xarray(pr, 0.95)

In [ ]:
# Add a dimension to be used as channel in the DNN
pr = pr.expand_dims('level', -1)
pr

In [ ]:
pr.tp[2,:,:].plot()

## Input data: meteorological fields

In [ ]:
# Load geopotential height
z = get_era5_data(PATH_ERA5 + '/geopotential/grid1/*.nc',
                  DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
z = z.sel(level=LEVELS)

# Get Z in geopotential height (m)
z.z.values = z.z.values/G

# Get axes
lats = z.lat
lons = z.lon

# Load temperature
t2m = get_era5_data(PATH_ERA5 + '/temperature/grid1/Grid1_*.nc',
                    DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
t2m['time'] = pd.DatetimeIndex(t2m.time.dt.date)
t2m = t2m.rename_vars({'T2MMEAN': 't'})

# Load relative humidity
rh = get_era5_data(PATH_ERA5 + '/relative_humidity/day_grid1/*.nc',
                   DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
rh['time'] = pd.DatetimeIndex(rh.time.dt.date)
rh = rh.sel(level=LEVELS)

# Load wind components
u850 = get_era5_data(PATH_ERA5 + '/U_wind/day_grid1/*.nc',
                     DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
u850['time'] = pd.DatetimeIndex(u850.time.dt.date)
v850 = get_era5_data(PATH_ERA5 + '/V_wind/day_grid1/*.nc',
                     DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
v850['time'] = pd.DatetimeIndex(v850.time.dt.date)

# Checking dimensions
print('dimension of pr:', pr.dims)
print('dimension of z', z.dims)
print('dimension of t2m:', t2m.dims)
print('dimension of rh:', rh.dims)
print('dimension of u:', u850.dims)
print('dimension of v:', v850.dims)


In [ ]:
# Merge arrays
X = xr.merge([z, t2m, rh, u850, v850])
X

In [ ]:
# Invert lat axis if needed
if X.lat[0].values < X.lat[1].values:
    X = X.reindex(lat=list(reversed(X.lat)))

### Split data and transform

In [ ]:
# Split into training and test
X_train_full = X.sel(time=slice('{}-01-01'.format(YY_TRAIN[0]),
                                '{}-12-31'.format(YY_TRAIN[1])))
X_test = X.sel(time=slice('{}-01-01'.format(YY_TEST[0]),
                          '{}-12-31'.format(YY_TEST[1])))

pr_train_full = pr.sel(time=slice('{}-01-01'.format(YY_TRAIN[0]),
                                  '{}-12-31'.format(YY_TRAIN[1])))
pr_test = pr.sel(time=slice('{}-01-01'.format(YY_TEST[0]),
                            '{}-12-31'.format(YY_TEST[1])))

In [ ]:
# Create a data generator
dic = {'z': LEVELS,
       't': None,
       'r': LEVELS,
       'u': None,
       'v': None}

from utils.utils_ml import *

YY_VALID = 1985

dg_train = WeatherDataGenerator(X_train_full.sel(time=slice(f'{YY_TRAIN[0]}', f'{YY_VALID}')),
                                pr_train_full.sel(time=slice(f'{YY_TRAIN[0]}', f'{YY_VALID}')),
                                dic, batch_size=64, load=True)
dg_valid = WeatherDataGenerator(X_train_full.sel(time=slice(f'{YY_VALID+1}', f'{YY_TRAIN[1]}')),
                                pr_train_full.sel(time=slice(f'{YY_VALID+1}', f'{YY_TRAIN[1]}')),
                                dic, mean=dg_train.mean, std=dg_train.std,
                                batch_size=64, load=True)
dg_test = WeatherDataGenerator(X_test, pr_test, dic,
                               mean=dg_train.mean, std=dg_train.std,
                               batch_size=64, load=True, shuffle=False)


In [ ]:
i_shape = dg_train.X.shape[1:]
o_shape = dg_train.y.shape[1:]

print(f'X shape: {i_shape}')
print(f'y shape: {o_shape}')


In [ ]:
plt.imshow(np.mean(dg_train.y, axis=0))

In [ ]:
n_figs = len(dg_train.X[0,0,0,:])
ncols = 5
nrows = -(-n_figs // ncols)
fig, axes = plt.subplots(figsize=(24, 3*nrows), ncols=ncols, nrows=nrows)
for i in range(n_figs):
    i_row = i // ncols
    i_col = i % ncols
    im = axes[i_row, i_col].imshow(np.mean(dg_train.X[:,:,:,i], axis=0))
    fig.colorbar(im, ax=axes[i_row, i_col])


# Model creation

In [ ]:
class EDMF(tf.keras.Model):
    """Encoder decoder model factory."""

    def __init__(self, arch, input_size, output_size, for_extremes=False, latent_dim=128,
                 dropout_rate=0.2):
        super(EDMF, self).__init__()
        self.arch = arch
        self.input_size = list(input_size)
        self.output_size = list(output_size)
        self.for_extremes = for_extremes
        self.latent_dim = latent_dim
        self.dropout_rate = dropout_rate

        self.last_activation = 'relu'
        if for_extremes:
            self.last_activation = 'sigmoid'

        if arch == 'cnn-v1':
            self.create_cnnv1()
        elif arch == 'cnn-v2':
            self.create_cnnv2()
        elif arch == 'cnn-v3':
            self.create_cnnv3()
        else:
            raise('The architecture was not correctly defined')
            
        self.crop_output()
        
        
    def create_cnnv1(self):
        self.encoder = tf.keras.Sequential(
            [
                layers.InputLayer(input_shape=self.input_size),
                layers.Conv2D(32, 3, padding='same', activation='relu'),
                layers.BatchNormalization(),
                layers.MaxPooling2D(pool_size=2),
                layers.SpatialDropout2D(self.dropout_rate),
                layers.Conv2D(64, 3, padding='same', activation='relu'),
                layers.BatchNormalization(),
                layers.MaxPooling2D(pool_size=2),
                layers.SpatialDropout2D(self.dropout_rate),
                layers.Flatten(),
                layers.Dense(self.latent_dim, activation='sigmoid'),
            ]
        )

        next_size = self.get_size_for(stride_factor=4)

        self.decoder = tf.keras.Sequential(
            [
                layers.InputLayer(input_shape=(self.latent_dim,)),
                layers.Dense(np.prod(next_size), activation='relu'),
                layers.Reshape(target_shape=next_size),
                layers.Conv2DTranspose(64, 3, strides=2, padding='same', activation='relu'),
                layers.Conv2DTranspose(32, 3, strides=2, padding='same', activation='relu'),
                layers.Conv2DTranspose(1, 3, strides=1, padding='same', activation=self.last_activation),
            ]
        )

        
    def create_cnnv2(self):
        """ self.encoder = tf.keras.Sequential(
            [
                layers.InputLayer(input_shape=self.data_size),
                layers.Conv2D(8, 3, padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2D(8, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2D(16, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2D(16, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2D(32, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Flatten(),
                layers.Dense(self.latent_dim, activation='sigmoid', kernel_initializer='he_normal')
            ]
        )
        
        preflat_shape = self.encoder.layers[-2].input.get_shape().as_list()[1:]

        self.decoder = tf.keras.Sequential(
            [
                layers.InputLayer(input_shape=(self.latent_dim,)),
                layers.Dense(np.prod(preflat_shape), activation='relu', kernel_initializer='he_normal'),
                layers.Reshape(target_shape=preflat_shape),
                layers.Conv2DTranspose(16, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2DTranspose(16, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2DTranspose(16, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2DTranspose(8, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2D(1, 3, strides=(1,1), padding='same', activation='relu', kernel_initializer='he_normal')
            ]
        ) """
        
        
    def create_cnnv3(self):
        """ self.encoder = tf.keras.Sequential(
            [
                layers.InputLayer(input_shape=self.data_size),
                layers.Conv2D(8, 3, padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2D(8, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2D(16, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2D(16, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2D(32, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2D(32, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2D(64, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Flatten(),
                layers.Dense(self.latent_dim, activation='sigmoid', kernel_initializer='he_normal')
            ]
        )
        
        preflat_shape = self.encoder.layers[-2].input.get_shape().as_list()[1:]

        self.decoder = tf.keras.Sequential(
            [
                layers.InputLayer(input_shape=(self.latent_dim,)),
                layers.Dense(np.prod(preflat_shape), activation='relu', kernel_initializer='he_normal'),
                layers.Reshape(target_shape=preflat_shape),
                layers.Conv2DTranspose(32, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2DTranspose(32, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2DTranspose(16, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2DTranspose(16, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2DTranspose(16, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2DTranspose(8, 3, strides=(2,2), padding='same', activation='relu', kernel_initializer='he_normal'),
                layers.Conv2D(1, 3, strides=(1,1), padding='same', activation='relu', kernel_initializer='he_normal')
            ]
        ) """
        

    def get_size_for(self, stride_factor):
        next_shape = self.output_size.copy()
        next_shape[0] = int(np.ceil(next_shape[0]/stride_factor))
        next_shape[1] = int(np.ceil(next_shape[1]/stride_factor))

        return next_shape


    def crop_output(self):
        
        h, w = self.decoder.layers[-1].output.get_shape().as_list()[1:3]  # reconstructed width and hight
        h_tgt, w_tgt = self.output_size[:2]
        dh = h - h_tgt  # deltas to be cropped away
        dw = w - w_tgt

        # Add to decoder cropping layer and final reshaping
        self.decoder.add(layers.Cropping2D(cropping=((dh//2, dh-dh//2), (dw//2, dw-dw//2))))
        self.decoder.add(layers.Reshape(target_shape=self.output_size,))

        
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
    
    def encode(self, x):
        return self.encoder(x)

    
    def decode(self, z):
        return self.decoder(z)

In [ ]:
# Clear session and set tf seed
keras.backend.clear_session()
tf.random.set_seed(42)

# Hyperparameters
h_params = {'latent_dim': 128,
            'dropout_rate': 0.2}

# Build and train the model with the first architecture
cnnv1 = EDMF('cnn-v1', i_shape, o_shape, for_extremes=False, **h_params)

cnnv1.compile(optimizer='adam', loss='mse')


In [ ]:
history = cnnv1.fit(dg_train, validation_data=dg_valid, epochs=10)

In [ ]:
# Plot training evolution
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.show()